## Welcome to the data mining portion of the course
- This section of the course moves away from studying free text and will instead focus on more sturctured data analysis. 
- From now on the notebooks will use a language called R which was specifically created for data analytics. 
- Like the previous sections the code is being provided for your reference and it is not necessary to understand the specifics in order to progress through the remainder of the course. 

### In this notebook we will be covering 
1. How to read in a CSV file from github 
2. How to select variable columns from a data frame 
3. How to change data types
4. Replacing null numeric cells with the column mean
5. Replacing null categorical cells with a zero 
6. Replacing null cells using random forest imputation

## Please run the follow cell of code to load the video lecture for this lesson 

In [1]:
#install and load packages required to embed introductory video 
install.packages("IRdisplay")
library(IRdisplay)

## for full width
IRdisplay::display_html('<style>.container { width:100% !important; }</style>')

## this does not work with most sites due to same-origin policy, but no problem with local files   
IRdisplay::display_html('<iframe src="https://edpuzzle.com/embed/media/5eb9ce2d1f22443f2e11e7c7" width=700, height=500></iframe> ') 

package 'IRdisplay' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\Aaron\AppData\Local\Temp\RtmpuejAli\downloaded_packages


Warning message:
"package 'IRdisplay' was built under R version 3.6.3"

## Read in the CSV file
- The first step in many data mining projects is to read a data set into the data analytics software. 
- In this lesson we are loading an intensive care unit (ICU) CSV file into R. 
- In future lessons we will be using this data set and machine learning algorithms to predict hospital death in the ICU. 

In [2]:
#install and load packages required to read in the CSV file 
install.packages("readr")
install.packages("rlang")
library (readr)
library (rlang)


#location of raw csv file on github repository 
urlfile="https://raw.githubusercontent.com/e-cui/ENABLE-HiDAV-Online-Modules/master/Data%20Mining%20Modules/csv_files/training_v2.csv"

#Read the CSV file into a data frame called training_v2 
training_v2<-read_csv(url(urlfile))

#print
print("CSV file loaded successfully")

package 'readr' successfully unpacked and MD5 sums checked


Warning message:
"cannot remove prior installation of package 'readr'"Warning message in file.copy(savedcopy, lib, recursive = TRUE):
"problem copying C:\Users\Aaron\anaconda3\Lib\R\library\00LOCK\readr\libs\i386\librcon.lib to C:\Users\Aaron\anaconda3\Lib\R\library\readr\libs\i386\librcon.lib: Permission denied"Warning message in file.copy(savedcopy, lib, recursive = TRUE):
"problem copying C:\Users\Aaron\anaconda3\Lib\R\library\00LOCK\readr\libs\i386\readr.dll to C:\Users\Aaron\anaconda3\Lib\R\library\readr\libs\i386\readr.dll: Permission denied"Warning message:
"restored 'readr'"


The downloaded binary packages are in
	C:\Users\Aaron\AppData\Local\Temp\RtmpuejAli\downloaded_packages
package 'rlang' successfully unpacked and MD5 sums checked


Warning message:
"cannot remove prior installation of package 'rlang'"Warning message in file.copy(savedcopy, lib, recursive = TRUE):
"problem copying C:\Users\Aaron\anaconda3\Lib\R\library\00LOCK\rlang\libs\i386\rlang.dll to C:\Users\Aaron\anaconda3\Lib\R\library\rlang\libs\i386\rlang.dll: Permission denied"Warning message:
"restored 'rlang'"


The downloaded binary packages are in
	C:\Users\Aaron\AppData\Local\Temp\RtmpuejAli\downloaded_packages


Warning message:
"package 'readr' was built under R version 3.6.3"Warning message:
"package 'rlang' was built under R version 3.6.3"Parsed with column specification:
cols(
  .default = col_double(),
  ethnicity = col_character(),
  gender = col_character(),
  hospital_admit_source = col_character(),
  icu_admit_source = col_character(),
  icu_stay_type = col_character(),
  icu_type = col_character(),
  urineoutput_apache = col_logical(),
  apache_3j_bodysystem = col_character(),
  apache_2_bodysystem = col_character()
)
See spec(...) for full column specifications.
Warning message:
"41783 parsing failures.
 row                col           expected    actual         file
8119 urineoutput_apache 1/0/T/F/TRUE/FALSE 680.3136  <connection>
8120 urineoutput_apache 1/0/T/F/TRUE/FALSE 665.9712  <connection>
8121 urineoutput_apache 1/0/T/F/TRUE/FALSE 929.4048  <connection>
8122 urineoutput_apache 1/0/T/F/TRUE/FALSE 2869.9488 <connection>
8124 urineoutput_apache 1/0/T/F/TRUE/FALSE 1291.1616 <co

[1] "CSV file loaded successfully"


## Selecting variable columns from a data frame
- In this section we are reducing the number of columns to a more managable number to allow for easier data manipulation.
- Most of the variables selected in this step were chosen because they are part of the Apache algorithm which was developed to predict patient death in the ICU. 
- Some variables from the orginal Apache algorithm are being excluded due to missing information. 
- We have also selected some new variables to see if we can improve the performance of our model. 

In [3]:
#select variable columns from the training_v2 data frame into a new data frame called practice_csv
install.packages("dplyr")
library(dplyr)
practice_csv = select(training_v2, 4, 43, 36, 79, 61, 42, 123, 107, 115, 46, 28, 29, 5, 16, 6, 35, 20)
 
#create sepsis and cardiovascular diagnosis columns from the diagnosis column
practice_csv<- mutate(practice_csv, sepsis = ifelse(apache_2_diagnosis == '113', '1', '0'))
practice_csv<- mutate(practice_csv, cardiovascular_diagnosis = ifelse(apache_2_diagnosis == '114', '1', '0'))

#delete original diagnosis column
practice_csv = select(practice_csv, -17)

#print
print("variables selected, sepsis and cardiovascular diagnosis columns added, diagnosis column deleted")

package 'dplyr' successfully unpacked and MD5 sums checked


Warning message:
"cannot remove prior installation of package 'dplyr'"Warning message in file.copy(savedcopy, lib, recursive = TRUE):
"problem copying C:\Users\Aaron\anaconda3\Lib\R\library\00LOCK\dplyr\libs\i386\dplyr.dll to C:\Users\Aaron\anaconda3\Lib\R\library\dplyr\libs\i386\dplyr.dll: Permission denied"Warning message:
"restored 'dplyr'"


The downloaded binary packages are in
	C:\Users\Aaron\AppData\Local\Temp\RtmpuejAli\downloaded_packages


Warning message:
"package 'dplyr' was built under R version 3.6.3"
Attaching package: 'dplyr'

The following objects are masked from 'package:stats':

    filter, lag

The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union



[1] "variables selected, sepsis and cardiovascular diagnosis columns added, diagnosis column deleted"


## Changing data types of the selected variables 
- Before replacing nulls values with a measure of center the correct data type must be assigned to each column in our data frame. 
- Numeric columns can have a mean, a median, and a mode. 
- Ordinal columns can have a median and a mode. 
- Categorical columns only have a mode. 

In [4]:
#change categorical variables to factor data type 
practice_csv$hospital_death<- as.factor(practice_csv$hospital_death)
practice_csv$sepsis<- as.factor(practice_csv$sepsis)
practice_csv$cardiovascular_diagnosis<- as.factor(practice_csv$cardiovascular_diagnosis)
practice_csv$intubated_apache<- as.factor(practice_csv$intubated_apache)
practice_csv$gcs_eyes_apache<- as.factor(practice_csv$gcs_eyes_apache)
practice_csv$gcs_motor_apache<- as.factor(practice_csv$gcs_motor_apache)

#change to numeric variables to numeric data type 
practice_csv$temp_apache<- as.numeric(practice_csv$temp_apache)
practice_csv$map_apache<- as.numeric(practice_csv$map_apache)
practice_csv$h1_heartrate_max<- as.numeric(practice_csv$h1_heartrate_max)
practice_csv$d1_resprate_max<- as.numeric(practice_csv$d1_resprate_max)
practice_csv$d1_potassium_max<- as.numeric(practice_csv$d1_potassium_max)
practice_csv$d1_creatinine_max<- as.numeric(practice_csv$d1_creatinine_max)
practice_csv$d1_hematocrit_max<- as.numeric(practice_csv$d1_hematocrit_max)
practice_csv$sodium_apache<- as.numeric(practice_csv$sodium_apache)
practice_csv$wbc_apache<- as.numeric(practice_csv$wbc_apache)
practice_csv$age<- as.numeric(practice_csv$age)
practice_csv$pre_icu_los_days<- as.numeric(practice_csv$pre_icu_los_days)
practice_csv$bmi<- as.numeric(practice_csv$bmi)

#check all variable data types to make sure they are correct
str(practice_csv)

tibble [91,713 x 18] (S3: tbl_df/tbl/data.frame)
 $ hospital_death          : Factor w/ 2 levels "0","1": 1 1 1 1 1 1 1 1 2 1 ...
 $ temp_apache             : num [1:91713] 39.3 35.1 36.7 34.8 36.7 36.6 35 36.6 36.9 36.3 ...
 $ map_apache              : num [1:91713] 40 46 68 60 103 130 138 60 66 58 ...
 $ h1_heartrate_max        : num [1:91713] 119 114 96 100 89 83 79 118 82 96 ...
 $ d1_resprate_max         : num [1:91713] 34 32 21 23 18 32 38 28 24 44 ...
 $ sodium_apache           : num [1:91713] 134 145 NA NA NA 137 135 140 142 139 ...
 $ d1_potassium_max        : num [1:91713] 4 4.2 NA 5 NA 3.9 5 5.8 5.2 4.1 ...
 $ d1_creatinine_max       : num [1:91713] 2.51 0.71 NA NA NA 0.71 0.85 2.05 1.16 0.83 ...
 $ d1_hematocrit_max       : num [1:91713] 27.4 36.9 NA 34 NA 44.2 37.5 25.5 37.9 37.2 ...
 $ wbc_apache              : num [1:91713] 14.1 12.7 NA 8 NA 10.9 5.9 12.8 24.7 8.4 ...
 $ gcs_eyes_apache         : Factor w/ 4 levels "1","2","3","4": 3 1 3 4 NA 4 4 4 4 4 ...
 $ gcs_motor_a

## Replacing null values using measures of center (mean, median, mode)  
- When replacing a null value with a measure of center it is important to consider the distribution of the data. 
- When numeric data is evenly distributed the mean is the best measure of center but if the data is skewed then the median is a better choice. 

In [5]:
#replace some numeric variable nulls with the column mean 
practice_csv = transform(practice_csv, bmi = ifelse(is.na(bmi), mean(bmi, na.rm=TRUE), bmi))
practice_csv = transform(practice_csv, temp_apache = ifelse(is.na(temp_apache), mean(temp_apache, na.rm=TRUE), temp_apache))
practice_csv = transform(practice_csv, map_apache = ifelse(is.na(map_apache), mean(map_apache, na.rm=TRUE), map_apache))
practice_csv = transform(practice_csv, h1_heartrate_max = ifelse(is.na(h1_heartrate_max), mean(h1_heartrate_max, na.rm=TRUE), h1_heartrate_max))
practice_csv = transform(practice_csv, d1_resprate_max = ifelse(is.na(d1_resprate_max), mean(d1_resprate_max, na.rm=TRUE), d1_resprate_max))
practice_csv = transform(practice_csv, d1_potassium_max = ifelse(is.na(d1_potassium_max), mean(d1_potassium_max, na.rm=TRUE), d1_potassium_max))
practice_csv = transform(practice_csv, d1_creatinine_max = ifelse(is.na(d1_creatinine_max), mean(d1_creatinine_max, na.rm=TRUE), d1_creatinine_max))
practice_csv = transform(practice_csv, d1_hematocrit_max = ifelse(is.na(d1_hematocrit_max), mean(d1_hematocrit_max, na.rm=TRUE), d1_hematocrit_max))
practice_csv = transform(practice_csv, sodium_apache = ifelse(is.na(sodium_apache), mean(sodium_apache, na.rm=TRUE), sodium_apache))
practice_csv = transform(practice_csv, wbc_apache = ifelse(is.na(wbc_apache), mean(wbc_apache, na.rm=TRUE), wbc_apache))
practice_csv = transform(practice_csv, pre_icu_los_days = ifelse(is.na(pre_icu_los_days), mean(pre_icu_los_days, na.rm=TRUE), pre_icu_los_days))
practice_csv = transform(practice_csv, age = ifelse(is.na(age), mean(age, na.rm=TRUE), age))


#replace some categorical variable nulls with 0 (0 is the mode for these columns)
#gcs_motor_apache nulls replaced with a 1 (1 is the mode for this column) 
practice_csv$sepsis[is.na(practice_csv$sepsis)] <- 0
practice_csv$cardiovascular_diagnosis[is.na(practice_csv$cardiovascular_diagnosis)] <- 0
practice_csv$gcs_motor_apache[is.na(practice_csv$gcs_motor_apache)] <- 1
practice_csv$gcs_eyes_apache[is.na(practice_csv$gcs_eyes_apache)] <- 1

#print
print("Numeric nulls replaced with column mean, categorical nulls replaced with 0 or 1")

[1] "Numeric nulls replaced with column mean, categorical nulls replaced with 0 or 1"


## Replacing null values using a random forest algorithm 
- This algorithm makes multiple decision trees that are used to predict the missing information in our data set. 
- One advantage of using random forest for imputation (replacement of missing data) is that is works for both numeric and categorical data types.


In [6]:
#use a random forest algorithm to replace the remaining nulls
#random forest imputation works for replacing both categorical and numeric variables
#ntree is the number of trees, maxiter is the maximum number of iterations (repeats) 
#more trees and more iterations can increase accuracy but takes more time to run
install.packages("missForest")
install.packages("randomForest")
library(missForest)
library(randomForest)
set.seed(96) 
practice_csv.imp <- missForest(practice_csv, verbose = TRUE, maxiter = 2, ntree = 10)

#check imputed values
practice_csv.imp$ximp

#assign imputed data frame to a new data frame called t
t<- practice_csv.imp$ximp

package 'missForest' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\Aaron\AppData\Local\Temp\RtmpuejAli\downloaded_packages
package 'randomForest' successfully unpacked and MD5 sums checked


Warning message:
"cannot remove prior installation of package 'randomForest'"Warning message in file.copy(savedcopy, lib, recursive = TRUE):
"problem copying C:\Users\Aaron\anaconda3\Lib\R\library\00LOCK\randomForest\libs\i386\randomForest.dll to C:\Users\Aaron\anaconda3\Lib\R\library\randomForest\libs\i386\randomForest.dll: Permission denied"Warning message:
"restored 'randomForest'"


The downloaded binary packages are in
	C:\Users\Aaron\AppData\Local\Temp\RtmpuejAli\downloaded_packages


Warning message:
"package 'missForest' was built under R version 3.6.3"Loading required package: randomForest
Warning message:
"package 'randomForest' was built under R version 3.6.3"randomForest 4.6-14
Type rfNews() to see new features/changes/bug fixes.

Attaching package: 'randomForest'

The following object is masked from 'package:dplyr':

    combine

Loading required package: foreach
Warning message:
"package 'foreach' was built under R version 3.6.3"Loading required package: itertools
Warning message:
"package 'itertools' was built under R version 3.6.3"Loading required package: iterators
Warning message:
"package 'iterators' was built under R version 3.6.3"

  missForest iteration 1 in progress...done!
    estimated error(s): 0 0.0292384 
    difference(s): 0 0.0001163048 
    time: 4.09 seconds

  missForest iteration 2 in progress...done!
    estimated error(s): 0 0.02922057 
    difference(s): 0 0.0002017162 
    time: 4.04 seconds



hospital_death,temp_apache,map_apache,h1_heartrate_max,d1_resprate_max,sodium_apache,d1_potassium_max,d1_creatinine_max,d1_hematocrit_max,wbc_apache,gcs_eyes_apache,gcs_motor_apache,age,pre_icu_los_days,bmi,intubated_apache,sepsis,cardiovascular_diagnosis
0,39.3,40,119,34,134.0000,4.000000,2.51000,27.40000,14.10000,3,6,68.00000,0.541666667,22.73000,0,1,0
0,35.1,46,114,32,145.0000,4.200000,0.71000,36.90000,12.70000,1,3,77.00000,0.927777778,27.42000,0,0,0
0,36.7,68,96,21,137.9664,4.251594,1.48865,34.53197,12.13333,3,6,25.00000,0.000694444,31.95000,0,0,0
0,34.8,60,100,23,137.9664,5.000000,1.48865,34.00000,8.00000,4,6,81.00000,0.000694444,22.64000,1,0,0
0,36.7,103,89,18,137.9664,4.251594,1.48865,34.53197,12.13333,1,1,19.00000,0.073611111,29.18582,0,0,0
0,36.6,130,83,32,137.0000,3.900000,0.71000,44.20000,10.90000,4,6,67.00000,0.000694444,27.56000,0,0,0
0,35.0,138,79,38,135.0000,5.000000,0.85000,37.50000,5.90000,4,6,59.00000,0.000694444,57.45000,1,0,0
0,36.6,60,118,28,140.0000,5.800000,2.05000,25.50000,12.80000,4,6,70.00000,0.002083333,29.18582,0,1,0
1,36.9,66,82,24,142.0000,5.200000,1.16000,37.90000,24.70000,4,6,45.00000,0.009027778,29.18582,0,0,0
0,36.3,58,96,44,139.0000,4.100000,0.83000,37.20000,8.40000,4,6,50.00000,0.060416667,25.71000,0,0,0


## This concludes our introduction to data analytics 
- The following questions were addressed between this notebook and the complementary video lecture:
1. What is a CSV file and how is it loaded into R? 
2. What are some simple and more advanced methods that can be used to deal with nulls? 
3. How should improbable data be handled?   


## (Informational Only) Exporting a CSV file 
- This section was provided to give an example of how R code can be used to export a CSV file to a specific file location. 

In [ ]:
#this code writes a CSV file of the dataframe to a specified file path location 
write.csv(dataframe,"file_path", row.names = FALSE)

## Interactive Quiz
-Example of how the video will work 

In [ ]:
install.packages("IRdisplay")
library(IRdisplay)

## for full width
IRdisplay::display_html('<style>.container { width:100% !important; }</style>')

## this does not work with most sites due to same-origin policy, but no problem with local files   
IRdisplay::display_html('<iframe src="https://edpuzzle.com/embed/media/5eb9ce2d1f22443f2e11e7c7" width=700, height=500></iframe> ') 